In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
!pip install tensorflow-hub

  Using cached tensorflow_hub-0.12.0-py2.py3-none-any.whl (108 kB)


In [3]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('./naver_story_002.csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
index = df[(df['grade'] == 'Unknown')|(df['grade'] == 'NR')|(df['grade'] == 'PG')|(df['grade'] == 'PG-13')|(df['grade'] == 'R')|(df['grade'] == 'G')].index
df.drop(index, inplace=True)
df.grade = df.grade.replace({'전체 관람가':0, '12세 관람가':1, '15세 관람가':2,'청소년 관람불가':3})
X = df['story']
y = df['grade']

In [5]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess =  tf.compat.v1.Session()
K.set_session(sess)
sess.run(tf.compat.v1.global_variables_initializer())
sess.run(tf.compat.v1.tables_initializer())

2022-08-02 17:43:40.057224: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-08-02 17:43:40.065618: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [6]:
n_of_train = int(len(X) * 0.8)
n_of_test = int(len(X) - n_of_train)
X_train = X[:n_of_train] 
y_train = np.asarray(y[:n_of_train]) 
X_test = X[n_of_train:] 
y_test = np.asarray(y[n_of_train:]) 

In [7]:
X_train = np.asarray([' '.join(t.split()[0:200]) for t in X_train])
X_test = np.asarray([' '.join(t.split()[0:200]) for t in X_test])

In [8]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]

In [9]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(4, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
history = model.fit(X_train, y_train, epochs=6, batch_size=100)

Train on 25394 samples
Epoch 1/6
25394/25394 [==============================] - 3909s 154ms/sample - loss: 1.3024 - acc: 0.3889
Epoch 2/6
25394/25394 [==============================] - 3919s 154ms/sample - loss: 1.2662 - acc: 0.4103
Epoch 3/6
25394/25394 [==============================] - 3855s 152ms/sample - loss: 1.2517 - acc: 0.4182
Epoch 4/6
25394/25394 [==============================] - 3845s 151ms/sample - loss: 1.2426 - acc: 0.4245
Epoch 5/6
25394/25394 [==============================] - 3845s 151ms/sample - loss: 1.2324 - acc: 0.4331
Epoch 6/6
25394/25394 [==============================] - 3852s 152ms/sample - loss: 1.2218 - acc: 0.4382


In [11]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

/opt/homebrew/Caskroom/miniforge/base/envs/dl/lib/python3.8/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates



 테스트 정확도: 0.4829
